# Projeto: Classificação de Imagens (Gatos x Cachorros)
Este projeto utiliza Transfer Learning com o modelo MobileNetV2 para classificar imagens de gatos e cachorros.
O dataset utilizado foi o PetImages (Kaggle), e o treinamento foi feito no Google Colab com TensorFlow/Keras.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


instalando dependecias
Importação das bibliotecas

In [ ]:
!pip install tensorflow

import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models
from sklearn.metrics import confusion_matrix, classification_report


baixando/carregando modelo

In [ ]:
# Substitua pelo caminho do seu Drive
path_dataset = '/content/drive/MyDrive/dio/transfer_learning_imagens/kagglecatsanddogs_5340/PetImages'

# Cria datasets de treino, validação e teste
train_ds = image_dataset_from_directory(
    path_dataset,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(160, 160),
    batch_size=32
)

val_ds = image_dataset_from_directory(
    path_dataset,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(160, 160),
    batch_size=32
)

# Melhora o desempenho do input pipeline
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)


Construção do Modelo (Transfer Learning com MobileNetV2)

In [ ]:
base_model = MobileNetV2(input_shape=(160, 160, 3),
                         include_top=False,
                         weights='imagenet')

base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1, activation='sigmoid')  # Binário: gato (0) x cachorro (1)
])


Compilação e Treinamento

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_ds,
                    validation_data=val_ds,
                    epochs=5)  # ou mais, se necessário


Gráfico de Acurácia e Perda

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(len(acc))

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Treino')
plt.plot(epochs_range, val_acc, label='Validação')
plt.title('Acurácia')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Treino')
plt.plot(epochs_range, val_loss, label='Validação')
plt.title('Perda')
plt.legend()
plt.show()


Matriz de Confusão e Métricas

In [ ]:
# Gera previsões no conjunto de validação
y_true = []
y_pred = []

for images, labels in val_ds:
    preds = model.predict(images)
    y_true.extend(labels.numpy())
    y_pred.extend((preds > 0.5).astype(int).flatten())

# Matriz de confusão
cm = confusion_matrix(y_true, y_pred)
labels = train_ds.class_names

plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
plt.xlabel('Previsto')
plt.ylabel('Real')
plt.title('Matriz de Confusão')
plt.show()

# Relatório
print(classification_report(y_true, y_pred, target_names=labels))
